# Oracle Datatypes

## NUMBER(P,S)
**Syntax:** `NUMBER(P,S)` — **P**=total digits, **S**=digits after decimal. Digits before decimal = `P-S`.
Extra decimals are **rounded**; exceeding total digits raises **ORA-01438**.

In [ ]:
DROP TABLE test;

In [ ]:
CREATE TABLE test (sno NUMBER(7,2));

In [ ]:
SELECT table_name FROM user_tables WHERE table_name = 'TEST';

In [ ]:
DESC test;

### Valid inserts

Stored exactly; 5 digits before and 2 after the decimal.

In [ ]:
INSERT INTO test VALUES (12345.67);

In [ ]:
SELECT * FROM test;

In [ ]:
INSERT INTO test VALUES (123.6);

In [ ]:
SELECT * FROM test;

### Invalid inserts

**ORA-01438** Error

In [ ]:
INSERT INTO test VALUES (123456.7);

In [ ]:
SELECT * FROM test;

Accepted but **rounded** to `12345.68` because S=2 (depending on client, insert succeeds and value rounds).

In [ ]:
INSERT INTO test VALUES (12345.6789);

In [ ]:
SELECT * FROM test;

## NUMBER(P)
**Syntax:** `NUMBER(P)` — no scale, so Oracle stores whole numbers; decimals are **rounded to nearest integer**.

In [ ]:
DROP TABLE test;

In [ ]:
CREATE TABLE test (sno NUMBER(7));

In [ ]:
SELECT table_name FROM user_tables WHERE table_name = 'TEST';

In [ ]:
DESC test;

### Valid inserts

Rounded to `100` due to no scale.

In [ ]:
INSERT INTO test VALUES (99.9);

In [ ]:
SELECT * FROM test;

Exactly 7 digits → valid.

In [ ]:
INSERT INTO test VALUES (1234567);

In [ ]:
SELECT * FROM test;

### Invalid inserts

**ORA-01438** because 8 digits exceed precision 7.

In [ ]:
INSERT INTO test VALUES (12345678);

In [ ]:
SELECT * FROM test;

Rounded to `99` (demonstrates decimal rounding).

In [ ]:
INSERT INTO test VALUES (99.4);

In [ ]:
SELECT * FROM test;

## INTEGER and FLOAT
- `INTEGER` is a subtype of NUMBER with scale 0 (whole numbers).
- `FLOAT` is approximate (binary floating), precision optional.

In [ ]:
DROP TABLE test;

In [ ]:
CREATE TABLE test (i INTEGER, f FLOAT);

In [ ]:
SELECT table_name FROM user_tables WHERE table_name = 'TEST';

In [ ]:
DESC test;

### Valid inserts

`i` stores 42; `f` stores an approximate value close to 3.14159 (binary).

In [ ]:
INSERT INTO test (i,f) VALUES (42, 3.14159);

In [ ]:
SELECT * FROM test;

Large magnitude is allowed for `FLOAT`.

In [ ]:
INSERT INTO test (i,f) VALUES (0, 1e125);

In [ ]:
SELECT * FROM test;

For `i` Oracle rounds to 4 (INTEGER behaves like NUMBER with scale 0).

In [ ]:
INSERT INTO test (i,f) VALUES (3.7, 2.5);

In [ ]:
SELECT * FROM test;

May overflow depending on environment — demonstrates limits.

In [ ]:
INSERT INTO test (i,f) VALUES (1, 1e4000);

In [ ]:
SELECT * FROM test;

## BINARY_FLOAT and BINARY_DOUBLE
IEEE 754 single- and double-precision approximate numeric types.

In [ ]:
DROP TABLE test;

In [ ]:
CREATE TABLE test (bf BINARY_FLOAT, bd BINARY_DOUBLE);

In [ ]:
SELECT table_name FROM user_tables WHERE table_name = 'TEST';

In [ ]:
DESC test;

### Valid inserts

Stored using IEEE754, so some decimal fractions are not exact.

In [ ]:
INSERT INTO test VALUES (CAST(1.2345 AS BINARY_FLOAT), CAST(1.234567890123 AS BINARY_DOUBLE));

In [ ]:
SELECT * FROM test;

Demonstrates special values (Infinity/NaN).

In [ ]:
INSERT INTO test VALUES (BINARY_FLOAT_INFINITY, BINARY_DOUBLE_NAN);

In [ ]:
SELECT * FROM test;

### Invalid inserts

**ORA-01722** invalid number — cannot convert text to numeric.

In [ ]:
INSERT INTO test VALUES ('abc','xyz');

In [ ]:
SELECT * FROM test;

Very large single-precision may overflow to Infinity.

In [ ]:
INSERT INTO test VALUES (CAST(1e100 AS BINARY_FLOAT), 0);

In [ ]:
SELECT * FROM test;

## CHAR(size)
Fixed-length character; Oracle **blank-pads** to the declared size. Max 2000 bytes.

In [ ]:
DROP TABLE test;

In [ ]:
CREATE TABLE test (name CHAR(5));

In [ ]:
SELECT table_name FROM user_tables WHERE table_name = 'TEST';

In [ ]:
DESC test;

### Valid inserts

Stored as `'abc  '` (2 trailing spaces); length reported as 5.

In [ ]:
INSERT INTO test VALUES ('abc');

In [ ]:
SELECT name, LENGTH(name) AS len FROM test;

Implicit numeric→char conversion; value becomes `'123  '`.

In [ ]:
INSERT INTO test VALUES (TO_CHAR(123));

In [ ]:
SELECT name, LENGTH(name) AS len FROM test;

### Invalid inserts

**ORA-12899** value too large (length > 5).

In [ ]:
INSERT INTO test VALUES ('abcdef');

In [ ]:
SELECT * FROM test;

Empty string is **NULL** in Oracle (not an error, illustrates behavior).

In [ ]:
INSERT INTO test VALUES (NULL);

In [ ]:
SELECT * FROM test;

## VARCHAR2(size)
Variable-length character; no padding. Max 4000 bytes.

In [ ]:
DROP TABLE test;

In [ ]:
CREATE TABLE test (name VARCHAR2(5));

In [ ]:
SELECT table_name FROM user_tables WHERE table_name = 'TEST';

In [ ]:
DESC test;

### Valid inserts

Stored exactly `'abc'`; length 3.

In [ ]:
INSERT INTO test VALUES ('abc');

In [ ]:
SELECT name, LENGTH(name) AS len FROM test;

Implicit timestamp→ Invalid value too large

In [ ]:
INSERT INTO test VALUES (TO_CHAR(SYSTIMESTAMP));

In [ ]:
SELECT name FROM test;

### Invalid inserts

**ORA-12899**: string longer than 5.

In [ ]:
INSERT INTO test VALUES ('abcdef');

In [ ]:
SELECT * FROM test;

Without TO_CHAR, Oracle attempts implicit conversion; may error if format model mismatches.

In [ ]:
INSERT INTO test VALUES (SYSTIMESTAMP);

In [ ]:
SELECT * FROM test;

## VARCHAR(size) — ANSI compatibility
Treated like `VARCHAR2` in Oracle; historically 2000-byte max.

In [ ]:
DROP TABLE test;

In [ ]:
CREATE TABLE test (name VARCHAR(5));

In [ ]:
SELECT table_name FROM user_tables WHERE table_name = 'TEST';

In [ ]:
DESC test;

### Valid inserts

Behaves like `VARCHAR2(5)`.

In [ ]:
INSERT INTO test VALUES ('xy');

In [ ]:
SELECT * FROM test;

Stores `'abc'`.

In [ ]:
INSERT INTO test VALUES ('abc');

In [ ]:
SELECT * FROM test;

### Invalid inserts

**ORA-12899**: too long for 5.

In [ ]:
INSERT INTO test VALUES ('abcdef');

In [ ]:
SELECT * FROM test;

Implicit date→char allowed; shows format dependence (use TO_CHAR).

In [ ]:
INSERT INTO test VALUES (DATE '2025-01-01');

In [ ]:
SELECT * FROM test;

## LONG 
Character data up to 2 GB; only one LONG column per table; 

In [ ]:
DROP TABLE test;

In [ ]:
CREATE TABLE test (notes LONG);

In [ ]:
SELECT table_name FROM user_tables WHERE table_name = 'TEST';

In [ ]:
DESC test;

### Valid inserts

Stores large text (demo is short).

In [ ]:
INSERT INTO test VALUES ('This is LONG data');

In [ ]:
SELECT * FROM test;

Second row inserted.

In [ ]:
INSERT INTO test VALUES ('More LONG data');

In [ ]:
SELECT * FROM test;

## NCHAR / NVARCHAR2 (national character set)
`NCHAR(size)` fixed-length; `NVARCHAR2(size)` variable-length Unicode.

In [ ]:
DROP TABLE test;

In [ ]:
CREATE TABLE test (c NCHAR(4), v NVARCHAR2(6));

In [ ]:
SELECT table_name FROM user_tables WHERE table_name = 'TEST';

In [ ]:
DESC test;

### Valid inserts

In [ ]:
INSERT INTO test (c,v) VALUES (N'ಕವಿ', N'हिंदी');

In [ ]:
SELECT c, v FROM test;

### Invalid inserts

In [ ]:
INSERT INTO test (c,v) VALUES (N'भारत', N'संस्कृत');

In [ ]:
SELECT * FROM test;

Implicit numeric→char conversions

In [ ]:
INSERT INTO test (c,v) VALUES (123, 456);

In [ ]:
SELECT * FROM test;

## DATE
Stores date+time to seconds. Default display via `NLS_DATE_FORMAT`.

In [ ]:
DROP TABLE test;

In [ ]:
CREATE TABLE test (d DATE);

In [ ]:
SELECT table_name FROM user_tables WHERE table_name = 'TEST';

In [ ]:
DESC test;

### Valid inserts

ISO date literal; time defaults to 00:00:00.

In [ ]:
INSERT INTO test VALUES (DATE '2025-01-15');

In [ ]:
SELECT * FROM test;

`SYSTIMESTAMP` converted to DATE (drops fractional seconds and TZ).

In [ ]:
INSERT INTO test VALUES (CAST(SYSTIMESTAMP AS DATE));

In [ ]:
SELECT * FROM test;

In [ ]:
INSERT INTO test VALUES (TIMESTAMP '2025-01-01 10:00:00');

In [ ]:
SELECT * FROM test;

### Invalid inserts

Invalid day → **ORA-01830/01858**.

In [ ]:
INSERT INTO test VALUES ('32-DEC-25');

In [ ]:
SELECT * FROM test;

## TIMESTAMP
Date/time with fractional seconds; no time zone.

In [ ]:
DROP TABLE test;

In [ ]:
CREATE TABLE test (ts TIMESTAMP);

In [ ]:
SELECT table_name FROM user_tables WHERE table_name = 'TEST';

In [ ]:
DESC test;

### Valid inserts

Implicitly cast to TIMESTAMP (time zone dropped).

In [ ]:
INSERT INTO test VALUES (SYSTIMESTAMP);

In [ ]:
SELECT * FROM test;

Explicit literal with fractional seconds.

In [ ]:
INSERT INTO test VALUES (TIMESTAMP '2025-01-01 10:00:00.123456');

In [ ]:
SELECT * FROM test;

Implicit DATE→TIMESTAMP allowed (00:00:00); included to discuss conversion.

In [ ]:
INSERT INTO test VALUES (DATE '2025-01-01');

In [ ]:
SELECT * FROM test;

### Invalid inserts

Invalid month/day → **ORA-01843/01861**.

In [ ]:
INSERT INTO test VALUES ('2025-13-40 10:00:00');

In [ ]:
SELECT * FROM test;